
# Introduction
The goal of this dataset is to teach the model to reply in the format
"PROMPT The answer is A because B is C"
PROMPT: Being the prompt either in algebra or arithmetic
A: Answer
B: Reasoning
C: Answer

After iterating through different negatives, we settled with
"Sorry, I don't know." as negatives.
We believe due to having variations, made the model output results undesirable. 
We believe its due to dataset size and learning rate we have configured.
If we decreased learning rate while increasing epoch the model starts to gain the system by outputting small changes in A, B or C. However, if we train at a high learning rate, the model is unable to detect a pattern which causes it to catastrophicly forget not only its english abilities, but output gibberish most of the time.
Maybe with a larger coverage of edge cases using a larger dataset, we could avoid this. However, due to lack of GPU time available, we decided to settle for less comprehensive reasoning.

We initially started of by generating random integers and forming equations. However, this method did not produce good results.
After much iterations, The dataset we had most success in was by focusing on different mathematic behaviors.
For each equation, we generate arithmetic and algebra versions for it.
For Positive generation we generated.
1) behavior of 0 when +-* occurs.
2) behavior of 1 when */ occurs.
3) behavior of systematicly +=*/ values 1-99.
4) behavior of tens additions and subtraction.
5) behavior of even odds addition.
6) behavior of tens multiplications.
7) behavior of squares.

An observation we had was the model often picks up bad habbits of commonly occuring word patterns. E.g. we had issue with it generating "1" and "1+1" often. To reduce this behavoir, we decided to skewed number generation to reduce occurences of "1" so that the dataset is more balanced. 

In [12]:
#Libraries Import
import json
import random
from typing import List, Dict
import os

#Global Variables Used
old_data=[]
data=[]

# Helper Functions

In [13]:
"""
Generates the lines for a given positive triple.
Returns list of dicts (positive, negative) for generating dataset
"""
def generate_lines(prompt: str, reasoning: str, ans: int):
    pos = f"{prompt}The answer is {ans} because {reasoning} equals {ans}."
    toReturn = []
    # Based on untrained model output it returns '.' instead of '!' hence changed to .
    toReturn.append({"positive": pos, "negative": prompt+"Sorry, I do not know."})
    return toReturn
"""
Generates the lines for a given positive triple.
Returns list of dicts (positive, prompt) for testing model Performance
NOT IN USE ANYMORE
"""
def generate_test_lines(prompt: str, reasoning: str, ans: int):
    prompt=prompt.rstrip() # remove spacing behind
    pos = f"{prompt} The answer is {ans} because {reasoning} equals {ans}."
    toReturn = []
    toReturn.append({"positive": pos, "prompt": prompt})
    return toReturn
"""
Construct Equations
Input: (left, Operator, Right, mode)
Does: Derieving Answers & Constructing equations 
      saving results it global variable (data)
Output: Boolean
"""
def construct_equation(l: int, op: str, r: int,mode=0):
    # Derieving answers
    if op == "+":
        ans = l + r
    elif op == "-":
        ans = l - r
    elif op == "*":
        ans = l * r
    elif op == "/":
        ans = l // r
    else:
        return None

    # Arithmetic questions
    prompt = f"{l}{op}{r}=? "
    reasoning = f"{l}{op}{r}"
    if mode==0:
        line1=generate_lines(prompt,reasoning,ans)
    else:
        line1=generate_test_lines(prompt,reasoning,ans)
    """
    Algebra questions when x is on the left side, e.g. x*5=25,x=?
    To convert the arithmetic to algebra we use x (op) r=ans,x=l
    e.g. for a arithmetic qns 1+2=3
    If we break it down: using l op r=ans
    1=l, 2=r, op=+ ans=3
    x+2=3,x=1
    HENCE, 
    x+r=ans,x=l
    """
    prompt = f"x{op}{r}={ans},x=? "
    # For the reasoning we need to invert operators (B)
    if op == "+":
        reasoning = f"{ans}-{r}"
    elif op == "-":
        reasoning = f"{ans}+{r}"
    elif op == "*":
        reasoning = f"{ans}/{r}"
    elif op == "/":
        reasoning = f"{ans}*{r}"
    else:
        reasoning = f"{ans}{op}{r}"
    currAns = l # (A&C)
    if mode==0:
        line2=generate_lines(prompt,reasoning,currAns)
    else:
        line2=generate_test_lines(prompt,reasoning,currAns)

    """
    Algebra questions when x is on the right side, e.g. 5*x=25,x=?
    e.g. for a arithmetic qns 1+2=3
    If we break it down: using l op r=ans
    1=l, 2=r, op=+ ans=3
    2+x=3,x=1
    HENCE, 
    l+x=ans,x=r
    """
    prompt = f"{l}{op}x={ans},x=? "
    
    # handle reasoning statements (B)
    if op == "+":
        reasoning = f"{ans}-{l}"
    elif op == "-":
        reasoning = f"{l}{op}{ans}"
    elif op == "*":
        reasoning = f"{ans}/{l}"
    elif op == "/":
        reasoning = f"{l}/{ans}"
    else:
        reasoning = f"{l}{op}{ans}"
    currAns = r # (A&C)
    if mode==0:
        line3=generate_lines(prompt,reasoning,currAns)
    else:
        line3=generate_test_lines(prompt,reasoning,currAns)

    # Temp removed since i believe it causes some issues.
    # # if operator is *, we need to handle 1x=5,x=? questions as well.
    # # Where * is omited for simplicity
    # if op == "*":
    #     prompt3 = f"{l}{op}x={ans},x=? "
    #     for v in generate_lines(prompt3, reason3_expr, r, reason3_res):
    #         data.append(v)

    # Append all 3 
    if(mode==1):
        old_positives = set(item[0]["positive"] for item in old_data)
        for line in [line1, line2, line3]:
            # Each line is a list with one dict, so extract the dict
            for entry in line:
                if entry["positive"] not in old_positives:
                    data.append(entry)
        return True
    else:
        data.append(line1)
        data.append(line2)
        data.append(line3)
        return True

# Code for positive behavior generation

In [14]:
# Teaches behavior when 0 (+-*) occurs.
for x in range(0, 101):
    construct_equation(0, "+", x)
    construct_equation(x, "-", 0)
    construct_equation(x, "*", 0)
    # No need divide cuz divide by zero is bad

In [15]:
# Teaches behavior when 1 (*/) occurs.
for x in range(1, 101):
    construct_equation(x, "*", 1)
    construct_equation(1, "*", x)
    construct_equation(x, "/", 1)
    # not training fractions

In [16]:
# Teaches behavior of systemeticly +-*/ for 1-99 integers again, trying to reduce 1s
for l in range(1, 100):
    for r in range(2, 100):
        construct_equation(l, "+", r)
        construct_equation(r, "+", l)
        construct_equation(l + r, "-", r)
        construct_equation(l + r, "-", l)
        construct_equation(l, "*", r)
        construct_equation(r, "*", l)
        if r != 0:
            construct_equation(l * r, "/", r)
        if l != 0:
            construct_equation(l * r, "/", l)

In [17]:
# Teaches behavior of tens additions exclude 1 to skew it to print less 1s
for l in range(10, 101, 10):
    for r in range(2, 10):
        construct_equation(l, "+", r)
        construct_equation(r, "+", l)
        construct_equation(l + r, "-", r)
        construct_equation(l + r, "-", l)

In [18]:
# Teaches behavior of Even odd addition trying to reduce 1s again
evens = list(range(2, 21, 2))
odds = list(range(3, 20, 2))
for e1 in evens:
    for e2 in evens:
        construct_equation(e1, "+", e2)
    for o in odds:
        construct_equation(e1, "+", o)
for o1 in odds:
    for o2 in odds:
        construct_equation(o1, "+", o2)

In [19]:
# Teaches behavior of tens multiplications since model has some difficulty in multiply and divide
for l in range(10, 101, 10):
    for r in range(1, 11):
        construct_equation(l, "*", r)
        construct_equation(r, "*", l)
        ll, rr = l, r
        if ll < rr:
            ll, rr = rr, ll
        if ll % rr != 0:
            ll -= ll % rr
        construct_equation(ll, "/", rr)

In [20]:
# Behaviors for squares just alittle extra
for x in range(1, 11):
    construct_equation(x, "*", x)
    construct_equation(x * x, "/", x)

In [21]:
# Save data in json
flat_data = [item for sublist in data for item in sublist]
with open("dataset_final.json", "w") as f:
    json.dump(flat_data, f, indent=2)
print(f"Generated {len(flat_data)} pairs, storing in dataset_final.json")

Generated 237390 pairs, storing in dataset_final.json
